#Agentic Workflow with Google GenAI Function Calling
Made by: Wilfredo Aaron Sosa Ramos (AI Lab Manager at RealityAI Labs)

In [1]:
!pip install -q google-generativeai google-ai-generativelanguage langchain langchain-community langchain-core langgraph langchain-google-genai

In [2]:
from google.colab import userdata
import os

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate

chat_google_genai = ChatGoogleGenerativeAI(model="gemini-2.0-flash-exp", temperature=0)

In [4]:
from pydantic import BaseModel, Field
from typing import List

class QuestionChoice(BaseModel):
    key: str = Field(description="A unique identifier for the choice using letters A, B, C, or D.")
    value: str = Field(description="The text content of the choice")
class MultipleChoiceQuestion(BaseModel):
    question: str = Field(description="The question text")
    choices: List[QuestionChoice] = Field(description="A list of choices for the question, each with a key and a value")
    answer: str = Field(description="The key of the correct answer from the choices list")
    explanation: str = Field(description="An explanation of why the answer is correct")

class OpenEndedQuestion(BaseModel):
    question: str = Field(description="The open-ended question text")
    answer: str = Field(description="The expected correct answer")
    feedback: List[str] = Field(description="A list of possible answers for the provided question")

class QuestionList(BaseModel):
    multiple_choice_questions: List[MultipleChoiceQuestion]
    open_ended_questions: List[OpenEndedQuestion]

In [5]:
def adapt_questions_for_json(output, parser):
    return {
        "questions": output,
        "format_instructions": parser.get_format_instructions()
    }


#1. Translation tool:

In [6]:
def translation_tool(user_query: str, source_lang: str, target_lang: str):
    """ Used for translating content from a source language to a target language. """
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                """You are a professional translator specializing in converting text into multiple languages with exceptional accuracy, cultural sensitivity, and tone preservation.
                  Follow these precise instructions to ensure high-quality translations:

                1. **Accuracy:** Reflect the exact meaning of the original text while adapting grammar and syntax to the target language.
                2. **Cultural Context:** Incorporate cultural nuances, idiomatic expressions, and regional variations relevant to the target audience.
                3. **Tone Consistency:** Maintain the tone and style of the original text, whether formal, technical, persuasive, or conversational.
                4. **Language Expertise:** Leverage advanced linguistic knowledge to handle complex terms, technical jargon, or ambiguous phrases appropriately.
                5. **Output Clarity:** Present translations cleanly and clearly, ensuring each language is distinct and readable."""
            ),
            (
                "human",
                """
                Here you have the content to translate: {user_query}
                You have to translate it from {source_lang} to {target_lang}

                Don't return the analysis, just return the final output

                """
            )
        ]
    )

    chain = prompt | chat_google_genai
    result = chain.invoke({
        "user_query": user_query,
        "source_lang": source_lang,
        "target_lang": target_lang
    })
    return result.content

#2. Summarization tool:

In [7]:
def summarization_tool(user_query: str, summarization_type: str):
    """ Used for summarize content"""
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                """
                You are a professional summarization specialist, responsible for condensing information into concise and coherent formats while preserving key details and maintaining contextual accuracy.
                Follow these guidelines for effective summarization:

                1. **Clarity and Brevity:** Eliminate unnecessary details while ensuring the summary is clear and directly conveys the main points.
                2. **Preserve Context:** Maintain the original meaning, emphasizing critical ideas, facts, or insights.
                3. **Flexible Formats:** Decide whether to present the summary as a single sentence, a paragraph, or bullet points, based on the content and what best conveys the information.
                If the user specifies a format, follow their instructions.
                4. **Audience Awareness:** Tailor the language, tone, and level of detail to suit the target audience.
                5. **Logical Flow:** Organize information logically, ensuring seamless understanding and flow.

                Below are examples demonstrating different summarization formats:

                ---

                Original text:
                "The recent advancements in artificial intelligence have led to significant breakthroughs in natural language processing. These include improved machine translation, more accurate
                speech recognition, and the ability to generate human-like text, transforming industries like healthcare, education, and customer service."

                **If single sentence format is best:**
                "Advancements in AI have revolutionized natural language processing, enabling better translation, speech recognition, and text generation across various industries."

                **If paragraph format is best:**
                "Recent progress in artificial intelligence has transformed natural language processing, with breakthroughs in machine translation, speech recognition, and text generation.
                These advancements are driving innovation in industries such as healthcare, education, and customer service."

                **If bullet points format is best:**
                - AI advancements revolutionize natural language processing.
                - Key breakthroughs:
                  - Enhanced machine translation.
                  - Improved speech recognition.
                  - Human-like text generation.
                - Impact on industries: healthcare, education, customer service.


                """
            ),
            (
                "human",
                """
                Here is the content that you must summarize: {user_query}
                And here is the summarization type: {summarization_type}
                You must summarize in the language of the given user query.
                """
            )
        ]
    )

    chain = prompt | chat_google_genai
    result = chain.invoke({
        "user_query": user_query,
        "summarization_type": summarization_type
    })
    return result.content

#3. Rewrite tool:

In [8]:
def rewrite_tool(user_query: str, focus: str, tone: str):
    """ Used for rewriting content"""
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                """
                You are a professional rewriter specializing in modifying text to improve clarity, adjust tone, or provide alternative phrasing.
                Use your expertise to decide whether to focus on clarity, tone adjustment, or alternative phrasing based on the content's context unless the user specifies their preference.
                Follow these guidelines:

                1. **Clarity and Precision:** Ensure the rewritten text is clear, concise, and easy to understand, eliminating ambiguities or overly complex language.
                2. **Tone Adaptation:** Adjust the tone to suit the context and audience, whether formal, professional, conversational, or creative.
                3. **Preserve Meaning:** Retain the original intent and key details while enhancing readability or style.
                4. **Alternative Phrasing:** Provide multiple phrasings for variety or emphasis if it improves the content's impact.
                5. **Audience-Centric:** Tailor the rewrite to meet the expectations and preferences of the target audience.

                Below are examples demonstrating how to rewrite content effectively:

                ---

                Original text:
                "The new policy aims to improve employee satisfaction by offering flexible work hours and remote work opportunities."

                **If clarity is the focus:**
                "The updated policy is designed to enhance employee satisfaction by introducing flexible work schedules and options for remote work."

                **If tone adjustment is required (formal):**
                "The revised policy seeks to promote greater employee satisfaction by implementing flexible working hours and opportunities for remote employment."

                **If tone adjustment is required (conversational):**
                "We’ve updated our policy to make employees happier, giving them more flexibility with work hours and the option to work remotely."

                **If alternative phrasing is appropriate:**
                1. "To boost employee satisfaction, the new policy includes flexible scheduling and remote work possibilities."
                2. "Flexible hours and remote work are key elements of the new policy aimed at enhancing employee happiness."
                """
            ),
            (
                "human",
                """
                Here is the content that you must rewrite: {user_query}
                Here is the focus for the rewriting task: {focus}
                And here is the tone for the rewriting task: {tone}
                You must rewrite in the language of the given user query.
                """
            )
        ]
    )

    chain = prompt | chat_google_genai
    result = chain.invoke({
        "user_query": user_query,
        "focus": focus,
        "tone": tone
    })
    return result.content

#4. Custom prompt tool:

In [9]:
def custom_prompt_tool(user_query: str, action: str):
    """ Used for managing custom prompts"""
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                """
                You are a professional assistant specializing in creating, managing, and reusing user-defined custom prompts. Your role is to facilitate the creation, saving,
                and application of custom prompts to ensure users can easily define and reuse specific instructions for consistent and efficient results. Follow these guidelines:

                1. **Define Custom Prompts:**
                  - Assist users in crafting clear, detailed, and purpose-driven custom prompts tailored to their needs.
                  - Ensure the prompts are actionable and cover all necessary instructions.

                2. **Save Prompts:**
                  - Label prompts descriptively for easy identification.
                  - Organize prompts into categories or tags if necessary for efficient retrieval.

                3. **Reuse Prompts:**
                  - Allow users to apply saved prompts directly to new tasks or content.
                  - Offer customization options to adapt previously saved prompts to new contexts.

                4. **User Preferences:**
                  - Default to user-defined preferences when saved prompts exist.
                  - Suggest improvements to prompts if they can be optimized for better performance.

                5. **Prompt History Management:**
                  - Maintain a record of frequently used prompts for quick access.
                  - Enable users to update, rename, or delete prompts as needed.
                """
            ),
            (
                "human",
                """
                Here is the custom prompt: {user_query}
                And here is the action that must be taken: {action}
                """
            )
        ]
    )

    chain = prompt | chat_google_genai
    result = chain.invoke({
        "user_query": user_query,
        "action": action
    })
    return result.content

#5. Question Generation tool:

In [10]:
from langchain_core.runnables.base import RunnableMap, RunnableLambda

def generate_questions_to_json_tool(user_query: str):
    """ Used for generating questions. The results are returned in a JSON format """
    prompt_generate_questions = ChatPromptTemplate.from_messages(
      [
          (
              "system",
              "You are a specialized creator of multiple choice and open-ended questions."
          ),
          ("human",
          """
            You are a professional in creating high-quality questions for assessments, discussions, or educational purposes. Based on the provided content, generate multiple-choice and/or free-response questions. Unless the user specifies otherwise, you should independently decide the type, number, and format of the questions. If the user provides specific instructions, prioritize their preferences over your decision-making. Follow these guidelines:

            1. **Clarity and Relevance:** Ensure each question is concise, focused, and directly aligned with the content.
            2. **Dynamic Formats:** Choose the type (multiple-choice or free-response) and number of questions based on the complexity and depth of the content. Use a combination of both formats when appropriate.
            3. **Multiple-Choice Questions:**
              - Include one correct answer and 2–4 plausible distractors.
              - Distractors should be logical, relevant, and neither overly obvious nor misleading.
            4. **Free-Response Questions:**
              - Use open-ended prompts that encourage thoughtful and detailed responses.
              - Focus on analytical, reflective, or application-based phrasing to elicit deeper understanding.
            5. **Challenge Level:** Adjust the difficulty of questions to match the intended audience’s knowledge level, ranging from basic recall to advanced critical thinking.

            Generate a question for the following query: {user_query}. Generate questions in the language of the given user query.
          """
          )
      ]
    )

    chain_generate_questions = prompt_generate_questions | chat_google_genai

    parser = JsonOutputParser(pydantic_object=QuestionList)
    prompt_questions_to_json = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a specialized creator of multiple choice and open-ended questions in JSON."
        ),
        ("human",
         """
         Please convert the following questions into JSON format:
          **Tasks:**
          1. For Multiple Choice Questions:
            - Include the question text.
            - Provide all choices, each with a unique key (e.g., A, B, C, D) and its corresponding value (text content).
            - Specify the correct answer using the key of the appropriate choice.
            - Add a brief explanation of why the answer is correct.

          2. For Open-Ended Questions:
            - Include the question text.
            - Provide the expected correct answer.
            - Add a list of possible feedback answers, reflecting different valid responses to the question.

          3. Ensure all questions are formatted as JSON objects and grouped in a list.

          **Input Questions:**
          {questions}

          **Output:**
          You must respond as a JSON object:
          {format_instructions}
         """
        )
    ]
    )

    chain_questions_to_json = prompt_questions_to_json | chat_google_genai
    combined_chain = (
        chain_generate_questions
        | RunnableLambda(lambda x: adapt_questions_for_json(x.content, parser))
        | chain_questions_to_json
        | parser
    )

    result = combined_chain.invoke(user_query)

    return result

In [25]:
generate_questions_to_json_tool("Large Language Models")

{'multiple_choice_questions': [{'question': 'Which of the following best describes a Large Language Model (LLM)?',
   'choices': [{'key': 'A',
     'value': 'A type of computer program that only understands numerical data.'},
    {'key': 'B',
     'value': 'A machine learning model trained on a massive dataset of text and code.'},
    {'key': 'C',
     'value': 'A simple algorithm that translates languages word-for-word.'},
    {'key': 'D',
     'value': 'A database that stores information about different languages.'}],
   'answer': 'B',
   'explanation': 'LLMs are machine learning models trained on vast amounts of text and code data, enabling them to understand and generate human-like text.'},
  {'question': 'What is a key characteristic of LLMs that allows them to generate human-like text?',
   'choices': [{'key': 'A',
     'value': 'Their ability to perform complex mathematical calculations.'},
    {'key': 'B',
     'value': 'Their use of pre-programmed rules for grammar and syntax.

#6. Default prompt:

In [11]:
def default_prompt_tool(user_query: str):
    """ Used for answering any kind of prompt that is not about Translation, Summarization, Rewriting, Question Generation or Custom Prompts"""
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                """
                You are the CoTeacher Assistant, designed to provide personalized and adaptive guidance to educators. Your role is to enhance the teaching experience by being context-aware
                and role-specific, tailoring your interactions to the needs of teachers. Based on the context provided in the interface, follow these professional guidelines:

                1. **Dynamic Adaptation:** Understand the user's specific teaching context, such as lesson planning, classroom management, or student engagement. Adapt your responses to provide
                highly relevant and actionable insights.

                2. **Role-Specific Support:** Serve as a collaborative partner in teaching, offering solutions that are practical, evidence-based, and aligned with best practices in education.

                3. **Clarity and Precision:** Ensure your responses are clear, concise, and directly address the user's inquiry or task.

                4. **Empathetic and Encouraging Tone:** Maintain a supportive and positive tone, empowering educators to achieve their goals with confidence.

                5. **Proactive Engagement:** Anticipate potential needs based on the user's input. For example:
                  - Offer tailored strategies for differentiated instruction.
                  - Suggest effective classroom management techniques.
                  - Provide constructive feedback on lesson plans.
                """
            ),
            (
                "human",
                """
                Here is the user's query: {user_query}
                """
            )
        ]
    )

    chain = prompt | chat_google_genai
    result = chain.invoke({
        "user_query": user_query
    })
    return result.content

In [12]:
import google.generativeai as genai
from google.ai.generativelanguage_v1beta.types import content
from google.colab import userdata

genai.configure(api_key=userdata.get('GOOGLE_API_KEY'))

In [16]:
tools = [
    translation_tool,
    summarization_tool,
    rewrite_tool,
    generate_questions_to_json_tool,
    custom_prompt_tool,
    default_prompt_tool
]

model = genai.GenerativeModel(model_name='gemini-2.0-flash-exp',
                              tools=tools)

In [17]:
model._tools.to_proto()

[function_declarations {
   name: "translation_tool"
   description: " Used for translating content from a source language to a target language. "
   parameters {
     type_: OBJECT
     properties {
       key: "user_query"
       value {
         type_: STRING
       }
     }
     properties {
       key: "target_lang"
       value {
         type_: STRING
       }
     }
     properties {
       key: "source_lang"
       value {
         type_: STRING
       }
     }
     required: "user_query"
     required: "source_lang"
     required: "target_lang"
   }
 }
 function_declarations {
   name: "summarization_tool"
   description: " Used for summarize content"
   parameters {
     type_: OBJECT
     properties {
       key: "user_query"
       value {
         type_: STRING
       }
     }
     properties {
       key: "summarization_type"
       value {
         type_: STRING
       }
     }
     required: "user_query"
     required: "summarization_type"
   }
 }
 function_declaration

#Examples (Without automating agentic workflow):

##Complex task: Translation + Question Generation

In [61]:
chat = model.start_chat()
response = chat.send_message("""Could you convert 'Large Language Models (LLMs) are advanced artificial intelligence systems trained on vast amounts of text data to understand and generate human-like language.
These models leverage deep learning techniques, particularly transformer architectures, to process and generate text across a wide range of contexts and tasks. LLMs are capable
of performing diverse functions, including language translation, summarization, content creation, and even complex problem-solving. Their capabilities continue to expand as they are fine-tuned
for specific applications, making them invaluable tools in industries such as healthcare, education, customer support, and software development.' into Spanish and after that creating questions from it?""")

In [62]:
response

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "function_call": {
                  "name": "translation_tool",
                  "args": {
                    "user_query": "Large Language Models (LLMs) are advanced artificial intelligence systems trained on vast amounts of text data to understand and generate human-like language.\\nThese models leverage deep learning techniques, particularly transformer architectures, to process and generate text across a wide range of contexts and tasks. LLMs are capable\\nof performing diverse functions, including language translation, summarization, content creation, and even complex problem-solving. Their capabilities continue to expand as they are fine-tuned\\nfor specific applications, making them invaluable tools in industries such as healthcare, education, customer supp

In [63]:
for part in response.parts:
    if fn := part.function_call:
        args = ", ".join(f"{key}={val}" for key, val in fn.args.items())
        print(f"{fn.name}({args})")

translation_tool(user_query=Large Language Models (LLMs) are advanced artificial intelligence systems trained on vast amounts of text data to understand and generate human-like language.\nThese models leverage deep learning techniques, particularly transformer architectures, to process and generate text across a wide range of contexts and tasks. LLMs are capable\nof performing diverse functions, including language translation, summarization, content creation, and even complex problem-solving. Their capabilities continue to expand as they are fine-tuned\nfor specific applications, making them invaluable tools in industries such as healthcare, education, customer support, and software development., target_lang=Spanish, source_lang=English)


In [64]:
fc = response.candidates[0].content.parts[0].function_call

In [65]:
fc

name: "translation_tool"
args {
  fields {
    key: "user_query"
    value {
      string_value: "Large Language Models (LLMs) are advanced artificial intelligence systems trained on vast amounts of text data to understand and generate human-like language.\\nThese models leverage deep learning techniques, particularly transformer architectures, to process and generate text across a wide range of contexts and tasks. LLMs are capable\\nof performing diverse functions, including language translation, summarization, content creation, and even complex problem-solving. Their capabilities continue to expand as they are fine-tuned\\nfor specific applications, making them invaluable tools in industries such as healthcare, education, customer support, and software development."
    }
  }
  fields {
    key: "target_lang"
    value {
      string_value: "Spanish"
    }
  }
  fields {
    key: "source_lang"
    value {
      string_value: "English"
    }
  }
}

In [69]:
result = translation_tool(user_query = fc.args['user_query'], source_lang = fc.args['source_lang'], target_lang = fc.args['target_lang'])

In [70]:
result

'Los Modelos de Lenguaje Grandes (LLM, por sus siglas en inglés) son sistemas avanzados de inteligencia artificial entrenados con grandes cantidades de datos de texto para comprender y generar lenguaje similar al humano. Estos modelos aprovechan técnicas de aprendizaje profundo, particularmente arquitecturas de transformadores, para procesar y generar texto en una amplia gama de contextos y tareas. Los LLM son capaces de realizar diversas funciones, incluyendo la traducción de idiomas, el resumen, la creación de contenido e incluso la resolución de problemas complejos. Sus capacidades continúan expandiéndose a medida que se ajustan para aplicaciones específicas, lo que los convierte en herramientas invaluables en industrias como la atención médica, la educación, la atención al cliente y el desarrollo de software.\n'

In [71]:
response = chat.send_message(
    genai.protos.Content(
    parts=[genai.protos.Part(
        function_response = genai.protos.FunctionResponse(
          name='translation_tool',
          response={'result': result}))]))

In [72]:
response

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "function_call": {
                  "name": "generate_questions_to_json_tool",
                  "args": {
                    "user_query": "Los Modelos de Lenguaje Grandes (LLM, por sus siglas en ingl\u00e9s) son sistemas avanzados de inteligencia artificial entrenados con grandes cantidades de datos de texto para comprender y generar lenguaje similar al humano. Estos modelos aprovechan t\u00e9cnicas de aprendizaje profundo, particularmente arquitecturas de transformadores, para procesar y generar texto en una amplia gama de contextos y tareas. Los LLM son capaces de realizar diversas funciones, incluyendo la traducci\u00f3n de idiomas, el resumen, la creaci\u00f3n de contenido e incluso la resoluci\u00f3n de problemas complejos. Sus capacidades contin\u00faan exp

In [73]:
for part in response.parts:
    if fn := part.function_call:
        args = ", ".join(f"{key}={val}" for key, val in fn.args.items())
        print(f"{fn.name}({args})")

generate_questions_to_json_tool(user_query=Los Modelos de Lenguaje Grandes (LLM, por sus siglas en inglés) son sistemas avanzados de inteligencia artificial entrenados con grandes cantidades de datos de texto para comprender y generar lenguaje similar al humano. Estos modelos aprovechan técnicas de aprendizaje profundo, particularmente arquitecturas de transformadores, para procesar y generar texto en una amplia gama de contextos y tareas. Los LLM son capaces de realizar diversas funciones, incluyendo la traducción de idiomas, el resumen, la creación de contenido e incluso la resolución de problemas complejos. Sus capacidades continúan expandiéndose a medida que se ajustan para aplicaciones específicas, lo que los convierte en herramientas invaluables en industrias como la atención médica, la educación, la atención al cliente y el desarrollo de software.)


In [74]:
fc = response.candidates[0].content.parts[0].function_call

In [75]:
fc

name: "generate_questions_to_json_tool"
args {
  fields {
    key: "user_query"
    value {
      string_value: "Los Modelos de Lenguaje Grandes (LLM, por sus siglas en inglés) son sistemas avanzados de inteligencia artificial entrenados con grandes cantidades de datos de texto para comprender y generar lenguaje similar al humano. Estos modelos aprovechan técnicas de aprendizaje profundo, particularmente arquitecturas de transformadores, para procesar y generar texto en una amplia gama de contextos y tareas. Los LLM son capaces de realizar diversas funciones, incluyendo la traducción de idiomas, el resumen, la creación de contenido e incluso la resolución de problemas complejos. Sus capacidades continúan expandiéndose a medida que se ajustan para aplicaciones específicas, lo que los convierte en herramientas invaluables en industrias como la atención médica, la educación, la atención al cliente y el desarrollo de software."
    }
  }
}

In [79]:
result = generate_questions_to_json_tool(user_query = fc.args['user_query'])

In [80]:
result

{'multiple_choice_questions': [{'question': '¿Qué son los Modelos de Lenguaje Grandes (LLM)?',
   'choices': [{'key': 'A', 'value': 'Sistemas de almacenamiento de datos.'},
    {'key': 'B',
     'value': 'Sistemas avanzados de inteligencia artificial entrenados con grandes cantidades de datos de texto.'},
    {'key': 'C', 'value': 'Programas de edición de texto.'},
    {'key': 'D', 'value': 'Herramientas de diseño gráfico.'}],
   'answer': 'B',
   'explanation': 'Los LLM son sistemas de IA entrenados con grandes cantidades de texto para entender y generar lenguaje.'},
  {'question': '¿Qué técnica de aprendizaje profundo utilizan principalmente los LLM?',
   'choices': [{'key': 'A', 'value': 'Redes neuronales convolucionales.'},
    {'key': 'B', 'value': 'Máquinas de vectores de soporte.'},
    {'key': 'C', 'value': 'Arquitecturas de transformadores.'},
    {'key': 'D', 'value': 'Algoritmos genéticos.'}],
   'answer': 'C',
   'explanation': 'Los LLM se basan principalmente en arquitectu

In [81]:
response = chat.send_message(
    genai.protos.Content(
    parts=[genai.protos.Part(
        function_response = genai.protos.FunctionResponse(
          name='generate_questions_to_json_tool',
          response={'result': result}))]))

In [83]:
response

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "Okay, I've translated the text to Spanish and generated some questions based on it. Here are the results:\n\n**Spanish Translation:**\n\nLos Modelos de Lenguaje Grandes (LLM, por sus siglas en ingl\u00e9s) son sistemas avanzados de inteligencia artificial entrenados con grandes cantidades de datos de texto para comprender y generar lenguaje similar al humano. Estos modelos aprovechan t\u00e9cnicas de aprendizaje profundo, particularmente arquitecturas de transformadores, para procesar y generar texto en una amplia gama de contextos y tareas. Los LLM son capaces de realizar diversas funciones, incluyendo la traducci\u00f3n de idiomas, el resumen, la creaci\u00f3n de contenido e incluso la resoluci\u00f3n de problemas complejos. Sus capacidades contin\u00faan 

In [84]:
response.text

"Okay, I've translated the text to Spanish and generated some questions based on it. Here are the results:\n\n**Spanish Translation:**\n\nLos Modelos de Lenguaje Grandes (LLM, por sus siglas en inglés) son sistemas avanzados de inteligencia artificial entrenados con grandes cantidades de datos de texto para comprender y generar lenguaje similar al humano. Estos modelos aprovechan técnicas de aprendizaje profundo, particularmente arquitecturas de transformadores, para procesar y generar texto en una amplia gama de contextos y tareas. Los LLM son capaces de realizar diversas funciones, incluyendo la traducción de idiomas, el resumen, la creación de contenido e incluso la resolución de problemas complejos. Sus capacidades continúan expandiéndose a medida que se ajustan para aplicaciones específicas, lo que los convierte en herramientas invaluables en industrias como la atención médica, la educación, la atención al cliente y el desarrollo de software.\n\n**Generated Questions:**\n\nHere ar

In [97]:
for part in response.parts:
    if fn := part.function_call:
        args = ", ".join(f"{key}={val}" for key, val in fn.args.items())
        print(f"{fn.name}({args})")

#AUTOMATED AGENTIC WORKFLOW:

In [13]:
function_map = {
    "translation_tool": translation_tool,
    "summarization_tool": summarization_tool,
    "rewrite_tool": rewrite_tool,
    "generate_questions_to_json_tool": generate_questions_to_json_tool,
    "custom_prompt_tool": custom_prompt_tool,
    "default_prompt_tool": default_prompt_tool
}

In [19]:
def run_generic_assistant(user_query: str):
  chat = model.start_chat()

  response = chat.send_message(user_query)

  result = None

  while True:
      for part in response.parts:
          if fn := part.function_call:
              args = ", ".join(f"{key}={val}" for key, val in fn.args.items())
              print(f"USING THE FUNCTION: {fn.name}({args})")
              break
      else:
          print(f"FINAL RESPONSE: {response.text}")
          break

      fc = response.candidates[0].content.parts[0].function_call
      result = function_map[fc.name](**fc.args)
      print(f"GENERATED RESULT: {result}")

      response = chat.send_message(
          genai.protos.Content(
              parts=[genai.protos.Part(
                  function_response=genai.protos.FunctionResponse(
                      name=fc.name,
                      response={'result': result}))]))

  return result, response.text

In [102]:
run_generic_assistant("""
Could you convert 'Large Language Models (LLMs) are advanced artificial intelligence systems trained on vast amounts of text data to understand and generate human-like language.
These models leverage deep learning techniques, particularly transformer architectures, to process and generate text across a wide range of contexts and tasks. LLMs are capable
of performing diverse functions, including language translation, summarization, content creation, and even complex problem-solving. Their capabilities continue to expand as they are fine-tuned
for specific applications, making them invaluable tools in industries such as healthcare, education, customer support, and software development.' into Spanish and after that creating questions from it?
""")

USING THE FUNCTION: translation_tool(user_query=Large Language Models (LLMs) are advanced artificial intelligence systems trained on vast amounts of text data to understand and generate human-like language.\nThese models leverage deep learning techniques, particularly transformer architectures, to process and generate text across a wide range of contexts and tasks. LLMs are capable\nof performing diverse functions, including language translation, summarization, content creation, and even complex problem-solving. Their capabilities continue to expand as they are fine-tuned\nfor specific applications, making them invaluable tools in industries such as healthcare, education, customer support, and software development., source_lang=English, target_lang=Spanish)
GENERATED RESULT: Los Modelos de Lenguaje Grandes (LLM, por sus siglas en inglés) son sistemas avanzados de inteligencia artificial entrenados con grandes cantidades de datos de texto para comprender y generar lenguaje similar al hu

{'multiple_choice_questions': [{'question': '¿Qué son los Modelos de Lenguaje Grandes (LLM)?',
   'choices': [{'key': 'A', 'value': 'Sistemas de almacenamiento de datos.'},
    {'key': 'B',
     'value': 'Sistemas avanzados de inteligencia artificial entrenados con grandes cantidades de datos de texto.'},
    {'key': 'C', 'value': 'Programas de edición de texto.'},
    {'key': 'D', 'value': 'Herramientas de diseño gráfico.'}],
   'answer': 'B',
   'explanation': 'Los LLM son sistemas de IA entrenados con grandes cantidades de texto para entender y generar lenguaje.'},
  {'question': '¿Qué técnica de aprendizaje profundo utilizan principalmente los LLM?',
   'choices': [{'key': 'A', 'value': 'Redes neuronales convolucionales.'},
    {'key': 'B', 'value': 'Máquinas de vectores de soporte.'},
    {'key': 'C', 'value': 'Arquitecturas de transformadores.'},
    {'key': 'D', 'value': 'Algoritmos genéticos.'}],
   'answer': 'C',
   'explanation': 'Los LLM se basan principalmente en arquitectu

In [118]:
run_generic_assistant("""
Could you summarize 'Retrieval-Augmented Generation (RAG) is a cutting-edge framework in artificial intelligence that combines information retrieval with generative AI models to enhance response
quality and relevance. In RAG, an AI system retrieves relevant documents or data from an external knowledge base in real-time and uses this retrieved information to inform the generation of responses.
This approach addresses a key limitation of generative models, which often rely solely on their pre-trained knowledge, potentially leading to outdated or inaccurate outputs. By integrating retrieval,
RAG ensures that responses are both contextually rich and factually accurate, making it particularly valuable for applications like customer support, research assistance, and content generation.'
then translate that summarization from English to Spanish and after that creating questions from it?
""")

USING THE FUNCTION: summarization_tool(user_query=Retrieval-Augmented Generation (RAG) is a cutting-edge framework in artificial intelligence that combines information retrieval with generative AI models to enhance response \nquality and relevance. In RAG, an AI system retrieves relevant documents or data from an external knowledge base in real-time and uses this retrieved information to inform the generation of responses.\nThis approach addresses a key limitation of generative models, which often rely solely on their pre-trained knowledge, potentially leading to outdated or inaccurate outputs. By integrating retrieval, \nRAG ensures that responses are both contextually rich and factually accurate, making it particularly valuable for applications like customer support, research assistance, and content generation., summarization_type=default)
GENERATED RESULT: Retrieval-Augmented Generation (RAG) enhances AI response quality by combining information retrieval with generative models. RAG

{'multiple_choice_questions': [{'question': '¿Cuál es el propósito principal de la Generación Aumentada por Recuperación (RAG)?',
   'choices': [{'key': 'A',
     'value': 'Limitar la cantidad de datos que utiliza un modelo de IA.'},
    {'key': 'B',
     'value': 'Mejorar la calidad de las respuestas de la IA mediante la combinación de recuperación de información y modelos generativos.'},
    {'key': 'C',
     'value': 'Aumentar la velocidad de procesamiento de los modelos de IA.'},
    {'key': 'D',
     'value': 'Reducir la necesidad de entrenamiento previo en modelos de IA.'}],
   'answer': 'B',
   'explanation': 'RAG aims to enhance the quality of AI responses by integrating information retrieval with generative models, allowing for more informed and contextually relevant outputs.'},
  {'question': '¿Cómo asegura RAG que las respuestas de la IA sean contextualmente ricas y precisas?',
   'choices': [{'key': 'A',
     'value': 'Utilizando únicamente el conocimiento pre-entrenado del

In [18]:
run_generic_assistant("""
Please, rewrite this: Retrieval-Augmented Generation (RAG) is a cutting-edge framework in artificial intelligence that combines information retrieval with generative AI models to
enhance response quality and relevance. In RAG, an AI system retrieves relevant documents or data from an external knowledge base in real-time and uses this retrieved information
to inform the generation of responses. This approach addresses a key limitation of generative models, which often rely solely on their pre-trained knowledge, potentially leading to
outdated or inaccurate outputs. By integrating retrieval, RAG ensures that responses are both contextually rich and factually accurate, making it particularly valuable for applications
like customer support, research assistance, and content generation. After that, translate it from English to German and finally summarize that content with bullet points.
""")

USING THE FUNCTION: rewrite_tool(tone=formal, focus=RAG, user_query=Retrieval-Augmented Generation (RAG) is a cutting-edge framework in artificial intelligence that combines information retrieval with generative AI models to \nenhance response quality and relevance. In RAG, an AI system retrieves relevant documents or data from an external knowledge base in real-time and uses this retrieved information \nto inform the generation of responses. This approach addresses a key limitation of generative models, which often rely solely on their pre-trained knowledge, potentially leading to \noutdated or inaccurate outputs. By integrating retrieval, RAG ensures that responses are both contextually rich and factually accurate, making it particularly valuable for applications \nlike customer support, research assistance, and content generation.)
GENERATED RESULT: Retrieval-Augmented Generation (RAG) represents an advanced framework within the field of artificial intelligence. It integrates inform

'*   Retrieval-Augmented Generation (RAG) ist ein fortschrittlicher KI-Rahmen.\n*   RAG integriert Informationsabruf mit generativen KI-Modellen.\n*   Ziel ist die Verbesserung der Qualität und Relevanz generierter Antworten.\n*   Ein KI-System greift in Echtzeit auf externe Wissensspeicher zu.\n*   Abgerufene Informationen steuern die Antwortgenerierung.\n*   RAG mildert Einschränkungen generativer Modelle, die auf Trainingsdaten basieren.\n*   Echtzeit-Abruf sorgt für kontextuelle und faktische Genauigkeit.\n*   Vorteilhaft für Kundensupport, Forschungshilfe und Inhaltserstellung.\n'

In [20]:
result, response = run_generic_assistant("""
Please, rewrite this: Retrieval-Augmented Generation (RAG) is a cutting-edge framework in artificial intelligence that combines information retrieval with generative AI models to
enhance response quality and relevance. In RAG, an AI system retrieves relevant documents or data from an external knowledge base in real-time and uses this retrieved information
to inform the generation of responses. This approach addresses a key limitation of generative models, which often rely solely on their pre-trained knowledge, potentially leading to
outdated or inaccurate outputs. By integrating retrieval, RAG ensures that responses are both contextually rich and factually accurate, making it particularly valuable for applications
like customer support, research assistance, and content generation. After that, translate it from English to German and finally summarize that content with bullet points.
""")

USING THE FUNCTION: rewrite_tool(tone=formal, focus=AI, user_query=Retrieval-Augmented Generation (RAG) is a cutting-edge framework in artificial intelligence that combines information retrieval with generative AI models to enhance response quality and relevance. In RAG, an AI system retrieves relevant documents or data from an external knowledge base in real-time and uses this retrieved information to inform the generation of responses. This approach addresses a key limitation of generative models, which often rely solely on their pre-trained knowledge, potentially leading to outdated or inaccurate outputs. By integrating retrieval, RAG ensures that responses are both contextually rich and factually accurate, making it particularly valuable for applications like customer support, research assistance, and content generation.)
GENERATED RESULT: Retrieval-Augmented Generation (RAG) represents an advanced framework within the field of artificial intelligence. It integrates information ret

In [21]:
result

'*   Retrieval-Augmented Generation (RAG) ist ein fortschrittlicher KI-Rahmen.\n*   RAG integriert Informationsabruf mit generativen KI-Modellen.\n*   Ziel ist die Verbesserung der Qualität und Relevanz generierter Antworten.\n*   Das System ruft dynamisch relevante Informationen aus einem externen Wissensspeicher ab.\n*   Diese Informationen steuern die Generierung von Antworten.\n*   RAG mildert Einschränkungen generativer Modelle, die auf Trainingsdaten beruhen.\n*   Es stellt sicher, dass Antworten kontextuell und faktisch präzise sind.\n*   Vorteilhaft für Kundensupport, Forschungshilfe und Inhaltserstellung.\n'

In [23]:
response

"Okay, here's the rewritten text, its translation to German, and a summary with bullet points:\n\n**Rewritten Text (Formal Tone):**\nRetrieval-Augmented Generation (RAG) represents an advanced framework within the field of artificial intelligence. It integrates information retrieval mechanisms with generative AI models to improve the quality and relevance of generated responses. In the RAG framework, an AI system dynamically retrieves pertinent documents or data from an external knowledge repository. This retrieved information is then utilized to guide the generation of responses. This methodology effectively mitigates a significant constraint of generative models, which typically depend exclusively on their pre-existing training data, potentially resulting in responses that are either outdated or factually incorrect. Through the incorporation of retrieval processes, RAG ensures that responses are both contextually comprehensive and factually precise. This makes it particularly advanta

In [24]:
result, response = run_generic_assistant("""
Generate tips for differentiated instruction
""")

USING THE FUNCTION: default_prompt_tool(user_query=Generate tips for differentiated instruction)
GENERATED RESULT: Okay, I can help with that! Here are some tips for differentiated instruction, tailored to help you meet the diverse needs of your students:

**Understanding Your Students**

1.  **Know Your Learners:**
    *   **Assess Regularly:** Use pre-assessments, formative assessments, and observations to understand students' prior knowledge, learning styles, and interests.
    *   **Individual Profiles:** Create student profiles that include learning preferences, strengths, and areas for growth.

**Strategies for Differentiation**

2.  **Content Differentiation:**
    *   **Vary Resources:** Provide a range of materials (e.g., texts at different reading levels, videos, audio recordings).
    *   **Tiered Content:** Offer different levels of complexity within the same topic.
    *   **Choice Boards:** Allow students to select from a variety of content options.

3.  **Process Differe

In [25]:
result

"Okay, I can help with that! Here are some tips for differentiated instruction, tailored to help you meet the diverse needs of your students:\n\n**Understanding Your Students**\n\n1.  **Know Your Learners:**\n    *   **Assess Regularly:** Use pre-assessments, formative assessments, and observations to understand students' prior knowledge, learning styles, and interests.\n    *   **Individual Profiles:** Create student profiles that include learning preferences, strengths, and areas for growth.\n\n**Strategies for Differentiation**\n\n2.  **Content Differentiation:**\n    *   **Vary Resources:** Provide a range of materials (e.g., texts at different reading levels, videos, audio recordings).\n    *   **Tiered Content:** Offer different levels of complexity within the same topic.\n    *   **Choice Boards:** Allow students to select from a variety of content options.\n\n3.  **Process Differentiation:**\n    *   **Flexible Grouping:** Use whole-class, small-group, and individual work based

In [26]:
response

"Okay, here are some tips for differentiated instruction:\n\n**Understanding Your Students**\n\n1.  **Know Your Learners:**\n    *   **Assess Regularly:** Use pre-assessments, formative assessments, and observations to understand students' prior knowledge, learning styles, and interests.\n    *   **Individual Profiles:** Create student profiles that include learning preferences, strengths, and areas for growth.\n\n**Strategies for Differentiation**\n\n2.  **Content Differentiation:**\n    *   **Vary Resources:** Provide a range of materials (e.g., texts at different reading levels, videos, audio recordings).\n    *   **Tiered Content:** Offer different levels of complexity within the same topic.\n    *   **Choice Boards:** Allow students to select from a variety of content options.\n\n3.  **Process Differentiation:**\n    *   **Flexible Grouping:** Use whole-class, small-group, and individual work based on learning needs.\n    *   **Learning Centers:** Set up stations with different ac